In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd 
import os 
import tensorflow as tf


In [1]:
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack

from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import os
import IPython
from IPython.display import Audio
import scipy
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.tree import export_graphviz
# import mglearn
import graphviz
import numpy as np
import pandas as pd
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
from keras.layers import Conv1D
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization,GlobalAveragePooling1D
from keras.layers.convolutional import MaxPooling1D

import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
# Importing the cv2 module
import cv2
from sklearn.model_selection import train_test_split

In [2]:
import librosa
import librosa.display

In [3]:
from os.path import dirname, join as pjoin
from scipy.io import wavfile
import scipy.io

In [4]:
fname="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/LABALA.csv"

os.path.isfile(fname)

True

In [5]:
meta_data=[]

In [6]:
#VERSION TEST DONNEES NON LABELLISEES

data_dir="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/data_emotions/audio_speech_actors/"



data_files=[]
file_names=[]


# Class=np.array(labels["Class"])
# fnames=np.array(labels["Filename"])



for filename in os.listdir(data_dir):
    f = os.path.join(data_dir,filename)
    data_files.append(f)
    file_names.append(filename)
    meta_data.append([f,filename])
        

data=pd.DataFrame(data_files,file_names)

In [7]:
import re



def condition(x):
    if x.startswith('03-01-03'):
        return 'Happy'
    elif x.startswith('03-01-04'):
        return 'Sad'
    elif x.startswith('03-01-05'):
        return 'Angry'
    elif x.startswith('03-01-06'):
        return 'Fearful'
    elif x.startswith('03-01-07'):
        return 'Disgust'
    else :
        return 'Surprised'
 

meta_data = []
for dirname, _, filenames in os.walk("C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/data_emotions/audio_speech_actors/"):
    for filename in filenames:
        meta_data.append([filename, os.path.join(dirname,filename)])
        
        
data = pd.DataFrame(meta_data, columns=['fichier','source'])

data['actor'] = data['source'].apply(lambda x : re.findall("\w+_\d+",x)[0])

data = data[data['actor']!="audio_speech_actors_01"]
data.reset_index(inplace=True,drop=True)

data['Classe'] = data['fichier'].apply(lambda x: re.split("-\d+\.wav",x)[0])

data['Sexe'] = data['actor'].apply(lambda x : 'Femelle' if int(x.split('_')[1])%2==0 else 'Mâle')

data['Labels'] = data['Classe'].apply(condition)

#Choice of two categories to compare

# dataH=data[data['Labels']=='Happy'].copy()   
# dataS=data[data['Labels']=='Sad'].copy()
# data=pd.concat([dataH,dataS])

# dataF=data[data['Labels']=='Fearful'].copy()   
# dataD=data[data['Labels']=='Disgust'].copy()
# data=pd.concat([dataF,dataD])

dataSUR=data[data['Labels']=='Surprised'].copy()   
dataANG=data[data['Labels']=='Anger'].copy()
data=pd.concat([dataSUR,dataANG])
data.reset_index()

fnames=np.array(data['fichier'])
actors=np.array(data['actor'])

                       

In [8]:
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")


Length= []

for i in range(len(fnames)):
    
    wav_fname = pjoin(data_dir,actors[i], fnames[i])
    samplerate, data_sample = wavfile.read(wav_fname)
    number_of_channels = 1
    length = data_sample.shape[0] / samplerate

    Data=np.array(data_sample).astype(float)
    
    lg = len(librosa.feature.spectral_bandwidth(y=Data)[0])
    Length.append(lg)

In [9]:
scaling=False

def scaler(array):
    scaler = MinMaxScaler()
    array=array.reshape(-1,1)
    scaler.fit(array)
    array=scaler.transform(array)
    return array

sample_length= []


MFCC_H=np.zeros((len(fnames),min(Length)))

spectral_bandwidth=np.zeros((len(fnames),min(Length)))
rms=np.zeros((len(fnames),min(Length)))
CONTRAST=np.zeros((len(fnames),min(Length)))


Lim = min(Length)

for i in range(len(fnames)):
    
    
    
    wav_fname = pjoin(data_dir,actors[i], fnames[i])
    samplerate, data_sample = wavfile.read(wav_fname)
    number_of_channels = 1
    length = data_sample.shape[0] / samplerate
    sample_length.append(length)
  
    Data=np.array(data_sample).astype(float)

    lib_STFT=np.abs(librosa.stft(Data)) 
    contrast=np.mean(librosa.feature.spectral_contrast(S=lib_STFT),axis=0)

    if scaling==True: 
        
        RMS= scaler(librosa.feature.rms(y=Data))
        SPC_BW= scaler(librosa.feature.spectral_bandwidth(y=Data)[0])
        mfcc_h = scaler(np.mean(librosa.feature.mfcc(y=Data),axis = 0))
        contrast=scaler(np.mean(librosa.feature.spectral_contrast(S=lib_STFT),axis=0))
        
    if scaling==False:
        
        RMS= librosa.feature.rms(y=Data)[0]
        SPC_BW= librosa.feature.spectral_bandwidth(y=Data)[0]
        mfcc_h = np.mean(librosa.feature.mfcc(y=Data),axis = 0)
        contrast=np.mean(librosa.feature.spectral_contrast(S=lib_STFT),axis=0)

    if len(mfcc_h)>min(Length):
        spectral_bandwidth[i]=SPC_BW[:Lim].flatten()
        MFCC_H[i]=mfcc_h[:Lim].flatten()

        
        rms[i]=RMS[:Lim].flatten()
        CONTRAST[i]=contrast[:Lim].flatten()
    else:
        
        spectral_bandwidth[i]=SPC_BW.flatten()
        MFCC_H[i]=mfcc_h.flatten()
        CONTRAST[i]=contrast.flatten()
        rms[i]=RMS.flatten()
        

In [10]:
print(MFCC_H.shape)
print(rms.shape)
print(spectral_bandwidth.shape)
print(CONTRAST.shape)

(477, 276)
(477, 276)
(477, 276)
(477, 276)


In [11]:
data_matrix=np.zeros((4,len(fnames),min(Length)))

data_matrix[0]=MFCC_H
data_matrix[1]=rms
data_matrix[2]=spectral_bandwidth
data_matrix[3]=CONTRAST

data_matrix=np.reshape(data_matrix,(len(fnames),Lim,4))

In [14]:
#MAIN ANALYSIS 6 EMOTIONS

Labels=data['Labels'].reset_index()

Num_labels= []

for i in range(len(fnames)):
#     if Labels["Labels"][i]=='Happy':
#         Num_labels.append(0)
#     if Labels["Labels"][i]=='Sad':
#         Num_labels.append(1)
    
#     if Labels["Labels"][i]=='Fearful':
#         Num_labels.append(0)
#     if Labels["Labels"][i]=='Disgust':
#         Num_labels.append(1)
    if Labels["Labels"][i]=='Angry':
        Num_labels.append(0)
    if Labels["Labels"][i]=='Surprised':
        Num_labels.append(1)        
        
#     if Labels[i]=='Angry':
#         Num_labels.append(2)
#     if Labels[i]=='Fearful':
#         Num_labels.append(3)
#     if Labels[i]=='Disgust':
#         Num_labels.append(4)
#     if Labels[i]=='Surprised':
#         Num_labels.append(5)



model = Sequential([

  layers.Conv1D(126, 1, padding='same', activation='relu', input_shape=(Lim,1)),
  layers.MaxPooling1D(),
  layers.Conv1D(64, 1, padding='same', activation='relu'),
  layers.MaxPooling1D(),

  layers.Conv1D(32, 1, padding='same', activation='relu'),
  layers.MaxPooling1D(),

  layers.Flatten(),
  layers.Dense(16, activation='relu'),
    
  layers.Dense(2,activation='softmax')
])


model.compile(optimizer='adam',loss='binary_crossentropy' )

# FULL EMOTIONS  TRAINING
    
x_train, x_test, y_train, y_test = train_test_split(MFCC_H, Num_labels, test_size=0.05, random_state=np.random.randint(500))
# x_train = x_train.to_numpy()int()
y_train = to_categorical(y_train)
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

#fitting data
model.fit(x_train, y_train,epochs=50,batch_size=64,validation_split=0.2)

#Obtain the accuracy of prediction for each class
# print(np.round(model.predict(x_test),1))



Epoch 1/50
6/6 [==============================] - 1s 54ms/step - loss: 0.6169 - val_loss: 0.0042
Epoch 2/50
6/6 [==============================] - 0s 34ms/step - loss: 0.0016 - val_loss: 5.2355e-04
Epoch 3/50
6/6 [==============================] - 0s 34ms/step - loss: 1.8719e-04 - val_loss: 2.4394e-05
Epoch 4/50
6/6 [==============================] - 0s 35ms/step - loss: 4.0335e-06 - val_loss: 4.5937e-07
Epoch 5/50
6/6 [==============================] - 0s 34ms/step - loss: 5.1220e-08 - val_loss: 4.1962e-08
Epoch 6/50
6/6 [==============================] - 0s 34ms/step - loss: 4.9830e-09 - val_loss: 1.0486e-08
Epoch 7/50
6/6 [==============================] - 0s 34ms/step - loss: 2.0656e-09 - val_loss: 4.7220e-09
Epoch 8/50
6/6 [==============================] - 0s 34ms/step - loss: 8.2208e-10 - val_loss: 2.9961e-09
Epoch 9/50
6/6 [==============================] - 0s 34ms/step - loss: 6.0238e-10 - val_loss: 2.3157e-09
Epoch 10/50
6/6 [==============================] - 0s 33ms/step - l

In [15]:

from keras.regularizers import l2
                 
Lambda= 3e-2                                           

# Build the Model
model = Sequential()

model.add(LSTM(input_shape=(Lim, 4), units=16,
               activation='tanh', 
               kernel_regularizer=l2(Lambda), recurrent_regularizer=l2(Lambda),
               return_sequences=True,
  
              ))

model.add(BatchNormalization())
model.add(LSTM(units=8,
               activation='tanh', 
               kernel_regularizer=l2(Lambda), recurrent_regularizer=l2(Lambda),
               return_sequences=True, 
                
              ))
model.add(BatchNormalization())
model.add(LSTM(units=8,
               activation='tanh', 
               kernel_regularizer=l2(Lambda), recurrent_regularizer=l2(Lambda),
               return_sequences=False, 

              ))
model.add(BatchNormalization())
model.add(Dense(units=2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

x_train, x_test, y_train, y_test = train_test_split(data_matrix, Num_labels, test_size=0.05, random_state=np.random.randint(500))


y_train = to_categorical(y_train)

model.fit(x_train, y_train,
                    epochs=100,
                    batch_size=126,
                    validation_split=0.2)

Epoch 1/100
3/3 [==============================] - 6s 636ms/step - loss: 3.0953 - accuracy: 0.4503 - val_loss: 2.9491 - val_accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 187ms/step - loss: 2.9771 - accuracy: 0.4448 - val_loss: 2.8858 - val_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 186ms/step - loss: 2.8741 - accuracy: 0.4475 - val_loss: 2.8243 - val_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 1s 178ms/step - loss: 2.7899 - accuracy: 0.4530 - val_loss: 2.7635 - val_accuracy: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 1s 189ms/step - loss: 2.7221 - accuracy: 0.4834 - val_loss: 2.7033 - val_accuracy: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 1s 178ms/step - loss: 2.6611 - accuracy: 0.5414 - val_loss: 2.6442 - val_accuracy: 0.0110
Epoch 7/100
3/3 [==============================] - 1s 186ms/step - loss: 2.6020 - accuracy: 0.5635 - val_loss: 2.5863 - val_accuracy: 

KeyboardInterrupt: 

In [16]:
print(to_categorical(y_test))
print(np.round(model.predict(x_test),1))

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
1/1 [==============================] - 1s 877ms/step
[[0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.3 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.3 0.6]
 [0.3 0.6]]


In [17]:
labelsMF=[]
for i in range(len(data)):
    if data['Sexe'].iloc[i]=='Mâle':
        labelsMF.append(0)
    else:
        labelsMF.append(1)

In [18]:
#Male VS Female RANDOMFOREST

# Good Performance for scaling == False !  90%~

x_train, x_test, y_train, y_test = train_test_split(MFCC_H, labelsMF, test_size=0.2, random_state=np.random.randint(500))

#print(y_train)

model = RandomForestClassifier(n_estimators=1000,
                               random_state=25)

    
model.fit(x_train, y_train)
prediction= model.predict(x_test)
f1= f1_score(y_test,prediction,average='weighted')


    
print(f1)
print(f1_score(y_test,prediction,average='micro'))
print(f1_score(y_test,prediction,average='macro'))


0.9062805208898536
0.90625
0.9062398263700489


In [28]:
scaling=False

def scaler(array):
    scaler = MinMaxScaler()
    array=array.reshape(-1,1)
    scaler.fit(array)
    array=scaler.transform(array)
    return array

sample_length= []


MFCC=np.zeros((len(fnames),Lim,20))
CONTRAST=np.zeros((len(fnames),Lim,7))


Lim = min(Length)

for i in range(len(fnames)):
    
    
    
    wav_fname = pjoin(data_dir,actors[i], fnames[i])
    samplerate, data_sample = wavfile.read(wav_fname)
    number_of_channels = 1
    length = data_sample.shape[0] / samplerate
    sample_length.append(length)
  
    Data=np.array(data_sample).astype(float)
    lib_STFT=np.abs(librosa.stft(Data)) 
    
    

    if scaling==True: 
        
        mfcc_h = librosa.feature.mfcc(y=Data)
        contrast=librosa.feature.spectral_contrast(S=lib_STFT)
        
        for k in range(20):
            
            mfcc_h[k] = scaler(mfcc_h[k]).flatten()
        for k in range(7):
            contrast[k] = scaler(contrast[k]).flatten()
    if scaling==False:

        mfcc_h = librosa.feature.mfcc(y=Data)
        contrast=librosa.feature.spectral_contrast(S=lib_STFT)


    if len(mfcc_h[0])>min(Length):
        
        aa=np.arange(Lim,len(mfcc_h[0]))
        contrast=np.delete(contrast,aa,axis=1).T
        mfcc_h=np.delete(mfcc_h,aa,axis=1).T
        MFCC[i] = np.vstack((mfcc_h))
        CONTRAST[i] = np.vstack((contrast))


    else:
         MFCC[i] = np.vstack((mfcc_h.T))
         CONTRAST[i] = np.vstack((contrast.T))
  

In [31]:

from keras.regularizers import l2

LAYERS = [8, 8, 8, 2]                
N = 20              
LR = 1e-2                   
LAMBD = 3e-2                      
DP = 0.0                         
RDP = 0.0                      
T=Lim

# Build the Model
model = Sequential()

model.add(LSTM(input_shape=(T, N), units=LAYERS[0],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())

model.add(LSTM(units=LAYERS[1],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())

model.add(LSTM(units=LAYERS[2],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=False, return_state=False,
               stateful=False, unroll=False
              ))

model.add(BatchNormalization())
model.add(Dense(units=LAYERS[3], activation='sigmoid'))

# Compile the model with Adam optimizer
model.compile(loss='binary_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

x_train, x_test, y_train, y_test = train_test_split(MFCC, Num_labels, test_size=0.05, random_state=np.random.randint(500))


y_train = to_categorical(y_train)

model.fit(x_train, y_train,
                    epochs=150,
                    batch_size=126,
                    validation_split=0.2)

Epoch 1/150
3/3 [==============================] - 6s 506ms/step - loss: 3.0686 - accuracy: 0.4828 - val_loss: 2.8877 - val_accuracy: 0.5205
Epoch 2/150
3/3 [==============================] - 1s 290ms/step - loss: 2.9790 - accuracy: 0.4793 - val_loss: 2.8391 - val_accuracy: 0.5205
Epoch 3/150
3/3 [==============================] - 1s 290ms/step - loss: 2.8848 - accuracy: 0.5276 - val_loss: 2.7921 - val_accuracy: 0.5205
Epoch 4/150
3/3 [==============================] - 1s 290ms/step - loss: 2.8367 - accuracy: 0.5483 - val_loss: 2.7465 - val_accuracy: 0.5205
Epoch 5/150
3/3 [==============================] - 1s 311ms/step - loss: 2.7714 - accuracy: 0.5586 - val_loss: 2.7021 - val_accuracy: 0.5205
Epoch 6/150
3/3 [==============================] - 1s 284ms/step - loss: 2.7051 - accuracy: 0.5552 - val_loss: 2.6585 - val_accuracy: 0.5205
Epoch 7/150
3/3 [==============================] - 1s 292ms/step - loss: 2.6476 - accuracy: 0.5517 - val_loss: 2.6154 - val_accuracy: 0.5205
Epoch 8/150
3

Epoch 59/150
3/3 [==============================] - 1s 290ms/step - loss: 1.2828 - accuracy: 0.6069 - val_loss: 1.3487 - val_accuracy: 0.5205
Epoch 60/150
3/3 [==============================] - 1s 290ms/step - loss: 1.2607 - accuracy: 0.6069 - val_loss: 1.3356 - val_accuracy: 0.5205
Epoch 61/150
3/3 [==============================] - 1s 293ms/step - loss: 1.2473 - accuracy: 0.6069 - val_loss: 1.3227 - val_accuracy: 0.5205
Epoch 62/150
3/3 [==============================] - 1s 291ms/step - loss: 1.2323 - accuracy: 0.6069 - val_loss: 1.3101 - val_accuracy: 0.5205
Epoch 63/150
3/3 [==============================] - 1s 289ms/step - loss: 1.2194 - accuracy: 0.6000 - val_loss: 1.2977 - val_accuracy: 0.5205
Epoch 64/150
3/3 [==============================] - 1s 290ms/step - loss: 1.2064 - accuracy: 0.6069 - val_loss: 1.2857 - val_accuracy: 0.5205
Epoch 65/150
3/3 [==============================] - 1s 290ms/step - loss: 1.1949 - accuracy: 0.6172 - val_loss: 1.2738 - val_accuracy: 0.5205
Epoch 